In [3]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import matplotlib.pyplot as plt

data = pd.read_csv('data/imputed_train.csv')
data['id'] = pd.to_datetime(data['id'])
data

,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25
0,2020-01-01 00:00:00,42.900000,0.718000,15.7,73.1,64.4
1,2020-01-01 01:00:00,33.600000,0.587000,10.1,74.8,66.0
2,2020-01-01 02:00:00,29.300000,0.400655,5.1,51.0,44.9
3,2020-01-01 03:00:00,30.500000,0.246000,7.2,27.7,25.1
4,2020-01-01 04:00:00,29.300000,0.204000,8.3,15.3,13.6
...,...,...,...,...,...,...
40986,2024-09-03 18:00:00,17.713860,0.222000,55.1,12.0,5.3
40987,2024-09-03 19:00:00,21.932757,0.245000,48.2,13.4,7.0
40988,2024-09-03 20:00:00,23.265996,0.234000,44.5,12.4,7.1
40989,2024-09-03 21:00:00,33.122175,0.225000,25.9,10.6,5.4


In [78]:
data_long = pd.melt(data, id_vars=['id'], value_vars=['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25'],
                    var_name='Pollutant', value_name='Value')
train_data = TimeSeriesDataFrame.from_data_frame(
    data_long,
    id_column="Pollutant",
    timestamp_column="id"
)
train_data.head()

Value
item_id    timestamp                 
valeur_NO2 2020-01-01 00:00:00   42.9
           2020-01-01 01:00:00   33.6
           2020-01-01 02:00:00   29.3
           2020-01-01 03:00:00   30.5
           2020-01-01 04:00:00   29.3

In [79]:
predictor = TimeSeriesPredictor(
    prediction_length=504,
    path="autogluon-m4-hourly",
    target="Value",
    eval_metric="MAE",
)

predictor.fit(
    train_data,
    presets="medium_quality",
    time_limit=600,
)

Beginning AutoGluon training... Time limit = 600s
AutoGluon will save models to 'autogluon-m4-hourly'
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          8
GPU Count:          0
Memory Avail:       11.00 GB / 31.70 GB (34.7%)
Disk Space Avail:   68.29 GB / 476.00 GB (14.3%)
Setting presets to: medium_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAE,
 'hyperparameters': 'light',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 504,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'target': 'Value',
 'time_limit': 600,
 'verbosity': 2}

Inferred time series frequency: 'H'
Provided train_data has 204955 rows, 5 time series. Median time series length is 40991 (min=40991, max=40991). 

Pro

In [83]:
prediction = predictor.predict(train_data)[["mean"]]
prediction

Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


mean
item_id     timestamp                     
valeur_NO2  2024-09-03 23:00:00  20.151454
            2024-09-04 00:00:00  19.795188
            2024-09-04 01:00:00  18.223992
            2024-09-04 02:00:00  17.992103
            2024-09-04 03:00:00  19.417732
...                                    ...
valeur_PM25 2024-09-24 18:00:00   8.706389
            2024-09-24 19:00:00   8.764595
            2024-09-24 20:00:00   9.172983
            2024-09-24 21:00:00   8.609866
            2024-09-24 22:00:00   7.961546

[2520 rows x 1 columns]

In [95]:
prediction_wide = prediction.reset_index().pivot(index='timestamp', columns='item_id', values='mean')
prediction_wide["id"] = pd.to_datetime(prediction_wide.index)

ImportError: cannot import name 'clean_for_submission' from 'utils.preprocessing' (c:\Users\User\Mon Drive\DSB\2A HEC\Time series\time_series_project\utils\preprocessing.py)

In [6]:
from utils.preprocessing import clean_for_submission

submission = pd.read_csv('submissions/matt_autogluon_raw.csv')
clean_for_submission(submission).to_csv('submissions/matt_autogluon_raw.csv', index=False)

# With all features

In [27]:
import importlib

import utils.preprocessing

importlib.reload(utils.preprocessing)

<module 'utils.preprocessing' from 'c:\\Users\\User\\Mon Drive\\DSB\\2A HEC\\Time series\\time_series_project\\utils\\preprocessing.py'>

In [28]:
from utils.preprocessing import preprocess_data
data = pd.read_csv('data/imputed_train.csv')
data = preprocess_data(data, add_filter_outliers=False, add_cyclic_features=False)
data = data[['id', 'valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10',
       'valeur_PM25', 'is_holiday', 'is_jour_ferie', 'precipitation',
       'wind_speed', 'temperature', 'humidity', 'pressure', 'visibility',
       'global_solar_radiation','is_weekend']]
data.head()

,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25,is_holiday,is_jour_ferie,precipitation,wind_speed,temperature,humidity,pressure,visibility,global_solar_radiation,is_weekend
0,2020-01-01 00:00:00,42.9,0.718000,15.7,73.1,64.4,1,0,0.0,1.5,0.8,94.0,1022.5,2139.0,0.0,0
1,2020-01-01 01:00:00,33.6,0.587000,10.1,74.8,66.0,1,0,0.0,2.6,-0.7,96.0,1022.2,509.0,0.0,0
2,2020-01-01 02:00:00,29.3,0.400655,5.1,51.0,44.9,1,0,0.0,1.9,-0.3,98.0,1022.2,779.0,0.0,0
3,2020-01-01 03:00:00,30.5,0.246000,7.2,27.7,25.1,1,0,0.0,1.8,0.0,98.0,1021.9,377.0,0.0,0
4,2020-01-01 04:00:00,29.3,0.204000,8.3,15.3,13.6,1,0,0.0,2.2,0.3,98.0,1021.6,1317.0,0.0,0


In [30]:
known_covariates = ['is_holiday', 'is_jour_ferie', 'precipitation',
       'wind_speed', 'temperature', 'humidity', 'pressure', 'visibility',
       'global_solar_radiation','is_weekend']

train_data = pd.melt(data, id_vars=['id'] + known_covariates, 
              value_vars=['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25'],
              var_name='Pollutant', value_name='Value')
train_data = TimeSeriesDataFrame.from_data_frame(
    train_data,
    id_column="Pollutant",
    timestamp_column="id",
)
train_data.head()

is_holiday  is_jour_ferie  precipitation  \
item_id    timestamp                                                       
valeur_NO2 2020-01-01 00:00:00           1              0            0.0   
           2020-01-01 01:00:00           1              0            0.0   
           2020-01-01 02:00:00           1              0            0.0   
           2020-01-01 03:00:00           1              0            0.0   
           2020-01-01 04:00:00           1              0            0.0   

                                wind_speed  temperature  humidity  pressure  \
item_id    timestamp                                                          
valeur_NO2 2020-01-01 00:00:00         1.5          0.8      94.0    1022.5   
           2020-01-01 01:00:00         2.6         -0.7      96.0    1022.2   
           2020-01-01 02:00:00         1.9         -0.3      98.0    1022.2   
           2020-01-01 03:00:00         1.8          0.0      98.0    1021.9   
           2020-01-01 04:00:00         2.2          0.3      98.0    1021.6   

                                visibility  global_solar_radiation  \
item_id    timestamp                                                 
valeur_NO2 2020-01-01 00:00:00      2139.0                     0.0   
           2020-01-01 01:00:00       509.0                     0.0   
           2020-01-01 02:00:00       779.0                     0.0   
           2020-01-01 03:00:00       377.0                     0.0   
           2020-01-01 04:00:00      1317.0                     0.0   

                                is_weekend  Value  
item_id    timestamp                               
valeur_NO2 2020-01-01 00:00:00           0   42.9  
           2020-01-01 01:00:00           0   33.6  
           2020-01-01 02:00:00           0   29.3  
           2020-01-01 03:00:00           0   30.5  
           2020-01-01 04:00:00           0   29.3

In [32]:
predictor = TimeSeriesPredictor(
    prediction_length=504,
    path="autogluon-m4-hourly",
    target="Value",
    eval_metric="MAE",
    known_covariates_names=known_covariates
)

predictor.fit(
    train_data,
    presets="high_quality",
    time_limit=900,
)

ValueError: Preset 'good_quality' was not found. Valid presets: ['best_quality', 'high_quality', 'medium_quality', 'fast_training']